# HST Clusters

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

# some basic packages
import os                 # filesystem related stuff
import json
from pathlib import Path  # use instead of os.path and glob
import sys                # mostly replaced by pathlib

import errno      # more detailed error messages
import warnings   # handles warnings
import logging    # use logging instead of print

from collections import OrderedDict  

# packages for scientific computing
import numpy as np
import scipy as sp

# packages for creating plots and figures
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# special functions for astronomy 
from astropy.table import Table  # useful datastructure
from astropy.table import vstack # combine multiple tables

from astropy.io import fits      # open fits files
from astropy.io import ascii     # handle normal files

from astropy.wcs import WCS               # handle coordinates
from astropy.coordinates import Angle     # work with angles (e.g. 1°2′3″)
from astropy.coordinates import SkyCoord  # convert pixel to sky coordinates
from astropy.visualization import simple_norm

from astropy.stats import sigma_clipped_stats  # calcualte statistics of images

import astropy.units as u        # handle units

from photutils import CircularAperture

from pymuse.constants import tab10, single_column, two_column
plt.style.use('TeX.mplstyle')

In [ ]:
logging.basicConfig(stream=sys.stdout,
                    #format='(levelname)s %(name)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

## Read in Data

### MUSE Data

In [ ]:
from pymuse.io import ReadLineMaps

name = 'NGC3351'

# first we need to specify the path to the raw data
data_raw = Path('d:\downloads\MUSEDAP')
basedir = Path('..')

# read in the data we will be working with and print some information
galaxy = ReadLineMaps(data_raw / name)



### Cluster Catalogue

In [ ]:
with fits.open(basedir / 'data' / 'external' / 'nebula_catalogue_FS_v01.fits') as hdul:
    nebulae = Table(hdul[1].data)

nebulae = nebulae[nebulae['gal_name']==name]    
nebulae.rename_columns(['cen_x','cen_y'],['x','y'])

### HST Data

In [ ]:
extern = Path('g:\Archive\HST')

filename = extern / name / 'clusters_photometry_v0_9' / f'{name}_phangshst_cluster_allcandidates_v0_9.csv'

if filename.is_file():
    clusters = ascii.read(filename)
else:
    raise FileExistsError('no catalogue for ' + name)
    
clusters['SkyCoord'] = SkyCoord(clusters['PHANGS_RA']*u.degree,clusters['PHANGS_DEC']*u.degree)
x,y= clusters['SkyCoord'].to_pixel(wcs=galaxy.wcs)
clusters['x'] = x
clusters['y'] = y
clusters.rename_columns(['PHANGS_MASS_MINCHISQ','PHANGS_AGE_MINCHISQ','PHANGS_EBV_MINCHISQ'],['mass','age','Ebv'])
clusters = clusters['x','y','mass','age','Ebv']
clusters['in_frame'] = False

y_dim,x_dim = galaxy.shape
for row in clusters:
    if 0<=int(row['x'])<x_dim and 0<=int(row['y'])<y_dim:
        if not np.isnan(galaxy.PSF[int(row['y']),int(row['y'])]):
            row['in_frame'] = True
            
clusters = clusters[clusters['in_frame'] & (clusters['mass']>1e3)]

In [ ]:
filename = extern / name / 'white_light_12nov2019' / f'{name}_white_24rgb.fits'

with fits.open(filename) as hdul:
    whitelight = hdul[0].data
    whitelight_header = hdul[0]

## Combine data

In [ ]:
from pymuse.plot.plot import create_RGB

HII = nebulae[nebulae['HII_region_class']==1]

fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot(projection=galaxy.wcs)

norm = simple_norm(galaxy.OIII5006_DAP,clip=False,percent=99)
#img = create_RGB(galaxy.HA6562,galaxy.OIII5006_DAP,galaxy.SII6716,weights=[0.6,1,0.6],percentile=[95,99.,95])
ax.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Blues)
#ax.imshow(img)

ax.scatter(clusters['x'],clusters['y'],marker='o',s=1,lw=0.3,edgecolor='orange',facecolors='none')
ax.scatter(HII['x'],HII['y'],marker='o',s=1,lw=0.3,edgecolor='red',facecolors='none')

print(f'{len(HII)} HII regions\n{len(clusters)} Clusters')

plt.savefig('clusters+HII.pdf',dpi=500)

In [ ]:
#img = create_RGB(galaxy.HA6562,galaxy.OIII5006_DAP,galaxy.SII6716,weights=[0.6,1,0.6],percentile=[95,99.,95])

fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot(projection=galaxy.wcs)

img = galaxy.OIII5006
norm = simple_norm(img,'linear',clip=False,percent=99)
ax.imshow(img,norm=norm,cmap=plt.cm.Greens)
ax.scatter(x,y,marker='o',s=1,lw=0.5,edgecolor='orange',facecolors='none')

PN = tbl[c_PN & c_limit]
ax.scatter(PN['x'],PN['y'],marker='o',s=3,lw=0.3,edgecolor='tab:red',facecolors='none')

plt.savefig('clusters.pdf',dpi=1000)
#norm = simple_norm(galaxy.OIII5006_DAP,clip=False,max_percent=95)
